In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

# Given data points
x_values_raw = np.array([-19, 1, -7, 8, -13, -14, 6, -6, 15, 18, 17, -2, 3, 5, 0, -3, -17, 16, 9, -5, 7, -9])
y_values_raw = np.array([44908454284646.23, -11.96, 2212419357.55, 6632284234.26, 1045333420142.03, 2127675836121.24, 371323111.36, 498723720.21, 3810778575297.02, 23129992889181.47, 13677923158831.61, 9908.02, 253157.75, 56108071.82, 3.83, 549470.91, 14991018277096.86, 7140198511573.15, 21459319022.04, 80415910.58, 1675302706.42, 27736872151.47])

# Combine x and y values into a single structure, then sort by x values
combined = np.column_stack((x_values_raw, y_values_raw))
sorted_combined = combined[np.argsort(combined[:, 0])]
print(sorted_combined)

# Extract sorted x and y values
x_values = sorted_combined[:, 0]
y_values = sorted_combined[:, 1]

# Prepare the design matrix for ridge regression and OLS
X_complete = np.vander(x_values, N=11, increasing=True)
y = y_values

# Define the range of lambda values for the optimization process
lambda_range = np.logspace(-5, 5, 200)
best_lambda = None
best_mse = np.inf

# Iterate over each lambda value to find the optimal one based on MSE
for lambda_val in lambda_range:
    ridge_model = Ridge(alpha=lambda_val, fit_intercept=False)
    ridge_model.fit(X_complete[:-1], y[:-1])  # Exclude the last observation from fitting
    y_pred = ridge_model.predict(X_complete[-1].reshape(1, -1))  # Predict for the excluded last observation
    mse = mean_squared_error([y[-1]], y_pred)  # Calculate MSE for the prediction
    
    # Update the best lambda if the current MSE is lower
    if mse < best_mse:
        best_mse = mse
        best_lambda = lambda_val

# Calculate OLS estimates for the complete dataset
alpha_OLS_complete = inv(X_complete.T @ X_complete) @ X_complete.T @ y

# Calculate Ridge Regression estimates using the optimal lambda for the complete dataset
alpha_ridge_optimal = inv(X_complete.T @ X_complete + best_lambda * np.eye(11)) @ X_complete.T @ y

# Generate a range of x values for plotting the polynomial fits
x_plot = np.linspace(min(x_values), max(x_values), 22)
X_plot = np.vander(x_values, N=11, increasing=True)

# Use the calculated coefficients to predict y values for both models
y_OLS_pred = X_complete @ alpha_OLS_complete
y_ridge_pred = X_complete @ alpha_ridge_optimal

# Plotting the actual data and the polynomial fits
plt.figure(figsize=(10, 6))
plt.scatter(x_values, y_values, color='blue', label='Actual Data')
plt.plot(x_plot, y_OLS_pred, label='OLS Fit', color='red')
plt.plot(x_values, y_ridge_pred, label=f'Ridge Fit (λ={best_lambda:.2e})', color='green')
plt.title('Polynomial Model Fits: OLS vs. Ridge Regression')
plt.xlabel('x')
plt.ylabel('f(x)')
plt.legend()
plt.grid(True)
plt.show()

alpha_OLS_complete,alpha_ridge_optimal,best_lambda
